In [1]:
#importações
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import win32com.client as win32
import time
import os
import pyautogui
import subprocess
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date, datetime, timedelta
#import foxesscloud.foxesscloud as f
import math
import glob

In [2]:
#função que retorna as datas
def datasmes(mode):

    # Obtenha a data atual
    data_atual = datetime.now()

    # Defina a data para o primeiro dia do mês (para atual/amterior é spo tirar/colocar "- relativedelta(months=1)" no final)

    primeiro_dia = data_atual.replace(day=1) 
    for _ in range (0, mode):
        primeiro_dia = (primeiro_dia.replace(day=1) - timedelta(days=1)).replace(day=1) 


    # Obtenha o último dia do mês 
    proximo_mes = primeiro_dia.month % 12 + 1
    novo_ano = primeiro_dia.year + (1 if proximo_mes == 1 else 0)
    ultimo_dia = (primeiro_dia.replace(month=proximo_mes, year=novo_ano) - timedelta(days=1))

    # Crie uma lista de datas com cada dia do mês 
    lista_de_datas = [primeiro_dia + timedelta(days=x) for x in range((ultimo_dia - primeiro_dia).days + 1)]

    # Formate as datas no formato 'dd/mm/yy'
    dat = [data.strftime('%d/%m/%y') for data in lista_de_datas]

    # Crie uma coluna com as datas no DataFrame
    df = pd.DataFrame({'Data': dat})
    
    
    return df, ultimo_dia.day, ultimo_dia.month, ultimo_dia.year, data_atual.strftime('%d-%m-%y')

In [3]:
def repair_and_save_excel(file_path):
    # Inicialize a aplicação Excel
    excel_app = win32.Dispatch('Excel.Application')
    excel_app.Visible = False  # Torne o Excel invisível

    try:
        # Abra o arquivo corrompido
        workbook = excel_app.Workbooks.Open(file_path)

        # Defina o novo caminho do arquivo
        new_file_path = os.path.splitext(file_path)[0] + '_repaired.xls'

        # Salve o arquivo como .xls
        workbook.SaveAs(new_file_path, FileFormat=56)  # 56 é o código para .xls

        # Feche o workbook
        workbook.Close()
        print(f"Arquivo salvo como {new_file_path}")

    except Exception as e:
        print(f"Erro ao tentar reparar o arquivo: {e}")

    finally:
        # Certifique-se de fechar o aplicativo Excel
        excel_app.Quit()

    return new_file_path

In [4]:
#Função para logar no site no SEMS
def logaSEMS(wait, navegador):
    navegador.get("https://www.semsportal.com/home/login")
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btnLogin"]')))
    navegador.find_element('xpath', '//*[@id="username"]').send_keys("energiasolar@holambra.com.br")
    navegador.find_element('xpath', '//*[@id="password"]').send_keys("Holambra2022")
    navegador.find_element('xpath', '//*[@id="readStatement"]').click()
    navegador.find_element('xpath', '//*[@id="btnLogin"]').click()
    time.sleep(2)
    
    return navegador.current_window_handle # Obter o identificador da guia atual

In [5]:
#função que armazena os dados de geração do mês do SEMS em um dataframe
def SEMS(uc, wait, navegador, download_path, main_tab, mode, mes_atual):
    element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div[1]/button/span')))
    navegador.find_element('xpath', '//*[@id="app"]/div[1]/div[2]/div/input').send_keys(uc)
    time.sleep(2)
    
    element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div[1]/button/span')))
    elemento = navegador.find_element('xpath', '//*[@id="app"]/div[1]/button/span')
    navegador.execute_script("arguments[0].click();", elemento)
    time.sleep(2)
    
    element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div[5]/div[2]/div/ul/li[2]/span[1]')))
    elemento = navegador.find_element('xpath', '//*[@id="app"]/div[5]/div[2]/div/ul/li[2]/span[1]')
    navegador.execute_script("arguments[0].click();", elemento)
    time.sleep(2)
    
    # Obter todos os identificadores de guia abertos
    all_tabs = navegador.window_handles
    
    if len(all_tabs) > 1:
        # Mudar o foco para a nova guia
        for tab in all_tabs:
            if tab != main_tab:
                navegador.switch_to.window(tab)
                break

        # Agora você está na nova guia, faça as operações necessárias aqui
        # Clica no botão "Generation & Income"
        time.sleep(2)
        element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[4]/div[2]/div/div[1]/div[2]/div[2]')))
        elemento = navegador.find_element('xpath', '/html/body/div[2]/div[4]/div[2]/div/div[1]/div[2]/div[2]')
        navegador.execute_script("arguments[0].click();", elemento)
        time.sleep(2)

        # Clica no botão exportar que leva pro relatório mensal
        element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[4]/div[2]/div/div[1]/div[3]')))
        elemento = navegador.find_element('xpath', '/html/body/div[2]/div[4]/div[2]/div/div[1]/div[3]')
        navegador.execute_script("arguments[0].click();", elemento)
        time.sleep(2)

        # Escolher o mês
        wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[3]/div/div[1]/div[4]/div[2]/div/input')))
        navegador.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[1]/div[4]/div[2]/div/input').click()
        time.sleep(1)

        if mode != 0:
            if mes_atual=='1':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[1]/td[1]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[1]/td[1]/a').click()
            elif mes_atual=='2':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[1]/td[2]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[1]/td[2]/a').click()
            elif mes_atual=='3':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[1]/td[3]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[1]/td[3]/a').click()
            elif mes_atual=='4':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[1]/td[4]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[1]/td[4]/a').click()
            elif mes_atual=='5':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[2]/td[1]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[2]/td[1]/a').click()
            elif mes_atual=='6':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[2]/td[2]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[2]/td[2]/a').click()
            elif mes_atual=='7':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[2]/td[3]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[2]/td[3]/a').click()
            elif mes_atual=='8':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[2]/td[4]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[2]/td[4]/a').click()
            elif mes_atual=='9':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[3]/td[1]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[3]/td[1]/a').click()
            elif mes_atual=='10':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[3]/td[2]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[3]/td[2]/a').click()
            elif mes_atual=='11':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[3]/td[3]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[3]/td[3]/a').click()
            elif mes_atual=='12':
                wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[3]/td[4]/a')))
                navegador.find_element(By.XPATH, '/html/body/div[8]/div[1]/div/div[2]/table[3]/tbody/tr[3]/td[4]/a').click()

        time.sleep(2)

        export = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[3]/div/div[1]/div[2]")))
        export.click()
        
        #localiza o caminho do ultimo arquivo baixado em Downloads
        time.sleep(10)  # Aguarda um tempo para garantir que o download foi concluído
        list_of_files = glob.glob(os.path.join(download_path, '*'))  # * significa todos os arquivos
        latest_file = max(list_of_files, key=os.path.getctime)

        # Usar openpyxl para arquivos xlsx
        df1 = pd.read_excel(latest_file, skiprows=20)  
        
        # Fechar a nova guia (opcional)
        navegador.close()

        # Voltar para a guia principal
        navegador.switch_to.window(main_tab)
        
        #Apaga a UC anterior
        element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div[1]/i')))
        elemento = navegador.find_element('xpath', '//*[@id="app"]/div[1]/i')
        navegador.execute_script("arguments[0].click();", elemento)
        time.sleep(2)
    
    return df1

In [6]:
#Função para logar no site no SOLARVIEW
def logaSOLARVIEW(wait, navegador, usuario, senha): 
    navegador.get("https://my.solarview.com.br/login/sair")
    iframe = navegador.find_element(By.ID,"temp_iframe")
    navegador.switch_to.frame(iframe)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="formLogin"]/div[4]/div/button')))
    navegador.find_element(By.ID, "email").send_keys(usuario)
    navegador.find_element(By.ID, "password").send_keys(senha)
    input("pressione enter quando terminar de fazer o capicha")
    #navegador.find_element('xpath', '//*[@id="formLogin"]/div[4]/div/button').click()
    #return navegador.current_window_handle # Obter o identificador da guia atual

In [7]:
#função que armazena os dados de geração do SOLARVIEW em um dataframe
def SOLARVIEW(uc, wait, navegador, main_tab, endereco, endexcel, vetor_erros, mode):

    time.sleep(2)
    navegador.switch_to.default_content()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="botaoMenu"]')))
    time.sleep(1)
    iframe = navegador.find_element(By.ID,"temp_iframe")
    navegador.switch_to.frame(iframe)

    #preenche UC no campo de busca
    #wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/div[1]/div[3]/div/div[1]/div/div/div/div/div/div/div/div[2]/ul/li[4]/button')))
    campo_busca = navegador.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[3]/div/div[3]/div/div[1]/div/div/div/input')
    campo_busca.send_keys(uc)
    campo_busca.send_keys(Keys.RETURN)
    time.sleep(5)

    #clicka na aba do grafico
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/div[1]/div[3]/div/div[2]/div/div/div/div/div[2]/div/div[1]/div[2]/div[1]/div/div[6]/div/div/div/button')))
    time.sleep(2)
    elemento = navegador.find_element('xpath', '//*[@id="main"]/div[1]/div[3]/div/div[2]/div/div/div/div/div[2]/div/div[1]/div[2]/div[1]/div/div[6]/div/div/div/button')
    navegador.execute_script("arguments[0].click();", elemento)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/div[1]/div[3]/div/div[2]/div/div/div/div/div[2]/div/div[1]/div[2]/div[1]/div/div[6]/div/div/div/div/button[2]')))
    time.sleep(2)
    try:
        navegador.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[3]/div/div[2]/div/div/div/div/div[2]/div/div[1]/div[2]/div[1]/div/div[6]/div/div/div/div/button[2]').click()
    except Exception:
        time.sleep(1)
        navegador.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[3]/div/div[2]/div/div/div/div/div[2]/div/div[1]/div[2]/div[1]/div/div[6]/div/div/div/div/button[2]').click()

    #muda para nova guia aberta e baixa o excel
    navegador.switch_to.default_content()
    janelas = navegador.window_handles
    navegador.switch_to.window(janelas[1])
    time.sleep(15)

    try:
        wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div/div[3]/div[3]/div/div/div')))
        navegador.find_element(By.XPATH, '/html/body/div[8]/div/div[3]/div[3]/div/div/div').click()     #clica no calendario para mudarmês
    
        time.sleep(5)
            
        #clica em mes atual se mode for 0 e mes passado para mode diferente
        if mode==0:
            wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[11]/div[3]/ul/li[5]')))
            navegador.find_element(By.XPATH, '/html/body/div[11]/div[3]/ul/li[5]').click() 
        elif mode==1:
            wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[11]/div[3]/ul/li[6]')))
            navegador.find_element(By.XPATH, '/html/body/div[11]/div[3]/ul/li[6]').click() 
        
        time.sleep(12)
        wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[8]/div/div[3]/div[4]/div/button/span')))
        navegador.find_element(By.XPATH, '/html/body/div[8]/div/div[3]/div[4]/div/button/span').click()
        time.sleep(12)  #aqui eu coloquei 20 segundos para dar tempo de baixar o arquivo
    
        #localiza o caminho do ultimo arquivo baixado em Downloads
        caminho_downloads = os.path.expanduser(endereco)
        arquivos_na_pasta = os.listdir(caminho_downloads)
        arquivos = [arquivo for arquivo in arquivos_na_pasta if os.path.isfile(os.path.join(caminho_downloads, arquivo))]
        arquivo_mais_recente = max(arquivos, key=lambda arquivo: os.path.getmtime(os.path.join(caminho_downloads, arquivo)))
        caminho_arquivo_mais_recente = os.path.join(caminho_downloads, arquivo_mais_recente)

        caminho_excel_reperado = repair_and_save_excel(caminho_arquivo_mais_recente)
        
        #tenta ler o arquivo baixado, se não printa erro
        try:
            xls = pd.ExcelFile(caminho_excel_reperado)
            if len(xls.sheet_names)>=2:
                df = pd.read_excel(caminho_excel_reperado, sheet_name=xls.sheet_names[1])
                df = df[['Data e Hora', 'Energia CA (kWh)']] 
            else:
                df = pd.read_excel(caminho_excel_reperado)
                df = df[['Data e Hora', 'Energia CA (kWh)']] 
        except Exception:
            print(" arquivo da uc ", uc, "corrompido \n" )
            df = pd.DataFrame()
            vetor_erros.append(uc)

    except Exception:
        print(" download da ", uc, "falhou \n" )
        df = pd.DataFrame()
        vetor_erros.append(uc)
    
    #fecha a aba atual e muda para principal
    navegador.close()
    janelas = navegador.window_handles
    navegador.switch_to.window(janelas[0])
    iframe = navegador.find_element(By.ID,"temp_iframe")
    navegador.switch_to.frame(iframe)

    #Apaga o campo de busca da UC anterior
    campo_busca = navegador.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[3]/div/div[3]/div/div[1]/div/div/div/input').clear()
    time.sleep(2)

    return df, vetor_erros

In [8]:
#função que transforma o dataframe do excel do solar view (df) em um df com os dados de cada dia do mês
def tratasview(df, dmes):
    if df.empty:
        novo_df = pd.DataFrame()
    
    else:
        df['Coluna']= df['Data e Hora'].str.slice(0, 2)      #cria uma coluna de indice com o dia do mês
        df['dif'] = df['Coluna'] != df['Coluna'].shift(1)    #compara se o index do dia é igual ao de cima
        df['dif'].fillna(True, inplace=True)                 #poe true na primeira linha que não tem de cima

        novo_df = df.loc[df['dif']].copy()                 #separa e copia apenas os dados diferentes
        novo_df['Energia CA (kWh)'] = novo_df['Energia CA (kWh)'].str.replace(',', '.')      #troca virgula por ponto
        novo_df['Energia CA (kWh)'] = novo_df['Energia CA (kWh)'].astype(float)              #converte em float

        novo_df['Energia CA (kWh)'] = novo_df['Energia CA (kWh)'].diff()                    #realiza a subtração
        novo_df = novo_df['Energia CA (kWh)']                                       #deixa apenas a coluna que importa
        novo_df = novo_df[1:]                                           #remove a primeira linha
        novo_df = novo_df.reset_index(drop=True)                        #reseta os index

        lf = dmes - len(novo_df)                                       #calcula linhas faltantes
        lz = pd.Series([0]* lf )       # Crie um Series com linhas contendo zeros
        novo_df = pd.concat([novo_df, lz], ignore_index=True)   #adiciona os zeros no novo_df
                            

        novo_df = pd.DataFrame({'qtd_Solarview': novo_df})   #transforma o vetor em um dataframe 

    return novo_df

In [10]:
# MAIN

#escolha do modo de mês que sera executado:
mode = 0 # escolher o mes / mes anterior
#mode = 0 # mes atual
antigo=''          #escolha o nome do arquivo antigo com base na data (ex: '29-12-23') deixar em branco ('') se não for comparar


#cria dataframes: (cuidado para não zerar os que você não que apagar!)
dfSEMS = pd.DataFrame()
dfsolarview = pd.DataFrame()     #cria df vazio para armazenar os solarview1, 2 e 3 e foxcloud
dfsolarview1 = pd.DataFrame()
dfsolarview2 = pd.DataFrame()
dfsolarview3 = pd.DataFrame()
dferros_df = pd.DataFrame()
vetor_erros = []

# Get the user's home directory
home = os.path.expanduser("~")
# Construct the path to the Downloads folder
download_path = os.path.join(home, "Downloads")

# Configuração do Selenium WebDriver
options = webdriver.ChromeOptions()

# Modificar o user agent
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
prefs = {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
options.add_experimental_option("prefs", prefs)

navegador = webdriver.Chrome(options=options)  #cria uma instância do driver do Chrome no Selenium

# Esperar a página carregar completamente
wait = WebDriverWait(navegador, 10)

#criando as variaveis e configurações:
#endexcel = "C:\\Program Files (x86)\\Microsoft Office\\root\\Office16\\EXCEL.EXE"   #caminho do excel de casa(Pedro)
#endexcel = "C:\\Program Files\\Microsoft Office\\root\\Office16\\EXCEL.EXE" #caminho do excel do trabalho    
endexcel = r"C:\Program Files\Microsoft Office\root\Office16\EXCEL.EXE"
dfUC = pd.DataFrame()                         #df que armazena os dados da UC atual
dfmain = pd.DataFrame()                       #df que armazena os dados de todas as UC até agora
datas, dmes, mmes, ymes, datatual= datasmes(mode)

# #----------------------------------------------------------------------//1-Primeira parte: isolarcloud//----------------------------------------------------------------------
# vetor_erros.append(21211) 

# #----------------------------------------------------------------------//7-Setima parte: Canadian Solar//----------------------------------------------------------------------
# UCANADIANSOLAR = [99807, 35198605, 5726700, 31383823, 31383807, 33112282, 18459960, 28158601, 18444377]
# vetor_erros += UCANADIANSOLAR   #adiciona as UC's no vetor_erros

# #----------------------------------------------------------------------//adicional-preenche dataframe com os erros//----------------------------------------------------------------------
# if vetor_erros:  
#     for uc in vetor_erros: 
#         UCS = [uc]*dmes
#         dfUC = pd.DataFrame({'UC': UCS}) 
#         dfUC['qtd_produzida'] = 0  
#         dfUC['dat_producao'] = datas                                       #loop que passa pelas unidades e armazena no dfmain
#         dferros_df = pd.concat([dferros_df, dfUC])
# else:
#     print("sem erros.")




In [11]:
#----------------------------------------------------------------------//parte: solarview1//----------------------------------------------------------------------
UCSOLARVIEW1 = [236502,444309,447802,447805,341602,1719610,37535315,34223002,100331,4001829031,4001833607,39170,79304,966302,33990,446204,446202]
#UCSOLARVIEW1 = [236502,444309,447802]
if UCSOLARVIEW1:    
    usuario = "energiasolar@holambra.com.br"
    senha ="Holambra2022"
    main_tab = logaSOLARVIEW(wait, navegador, usuario, senha)                                          #função que loga no site
    for uc in UCSOLARVIEW1:                                   #loop que passa pelas unidades e armazena no dfmain
        dfUC, vetor_erros = SOLARVIEW(uc, wait, navegador, main_tab, download_path, endexcel, vetor_erros, mode)
        dfUC = tratasview(dfUC, dmes)
        dfUC['dat_producao'] = datas                        #adiciona a coluna com as datas
        dfUC['UC'] = [uc]*dmes                              #adiciona a coluna com a UC
        dfsolarview1 = pd.concat([dfsolarview1, dfUC])      #adiciona dfUC no dfsolarview1

    dfsolarview = pd.concat([dfsolarview, dfsolarview1])    #armazena o dfsolarview1 no dfsolarview
else:
    print("Solarview1 vazio.")

dfsolarview = dfsolarview.fillna(0)
dfsolarview['qtd_Solarview'] = dfsolarview['qtd_Solarview'].astype(str)      #converte em string
dfsolarview['qtd_Solarview'] = dfsolarview['qtd_Solarview'].str.replace('.', ',')      #troca virgula por ponto
dfs1exp = pd.DataFrame()
dfs1exp['qtd_produzida'] = dfsolarview['qtd_Solarview']
dfrelacao = pd.read_excel('relacaoUC.xlsx')
dfs1exp = pd.merge(dfsolarview, dfrelacao)
dfs1exp.rename(columns={'qtd_Solarview': 'qtd_produzida'}, inplace = True)
dfs1exp['tip_origem'] =  'Gerada' #Cria coluna "tip_origem"
#dfs1exp['qtd_produzida'] = dfsolarview['qtd_Solarview']
dfs1exp = dfs1exp[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida']] 
dfs1exp.to_csv(f"mes_{mmes}_Geração_Solarview1.csv", sep=";", index=False )

Arquivo salvo como C:\Users\LucasLopesdeAlmeida\Downloads\ALBERTO NASCIMENTO - (HOLAMBRA) - U.C. 236502_2024-06-01_2024-06-30_repaired.xls


C:\Users\LucasLopesdeAlmeida\AppData\Local\Temp\ipykernel_10128\232531111.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['dif'].fillna(True, inplace=True)                 #poe true na primeira linha que não tem de cima


 download da  444309 falhou 

 download da  447802 falhou 

 download da  447805 falhou 

 download da  341602 falhou 

 download da  1719610 falhou 

 download da  37535315 falhou 

 download da  34223002 falhou 

 download da  100331 falhou 

 download da  4001829031 falhou 

 download da  4001833607 falhou 

 download da  39170 falhou 

 download da  79304 falhou 

 download da  966302 falhou 

 download da  33990 falhou 

 download da  446204 falhou 

 download da  446202 falhou 



In [12]:
UCSEMS = [100331,4001829031,39170,79304,966302,33990,446204,446206,34831878,37300873,31112304,4001833607,40832694,42401,230513,37300,79202,59391,59392,418306,403904,418301,956902,102604,28352386,29424720,40538818,3714918,44088752,446207,22404,446202,355704]
#UCSEMS = [34831878,37300873,100331]
if UCSEMS: 
    mes_des= '5' # Coloque o mes desejado
    main_tab = logaSEMS(wait, navegador)                                               #função que loga no site
    for uc in UCSEMS:                                        #loop que passa pelas unidades e armazena no dfmain
        dfUC = SEMS(uc, wait, navegador, download_path, main_tab, mode, mes_des)
        dfUC['UC'] = uc
        dfUC['dat_producao'] = datas
        dfUC['qtd_SEMS'] = dfUC['Geração de energia(kWh)']
        dfSEMS = pd.concat([dfSEMS, dfUC])
else:
    print("SEMS vazio.")

dfSEMS = dfSEMS.fillna(0)
dfSEMS['qtd_SEMS'] = dfSEMS['qtd_SEMS'].astype(str)      #converte em string
dfSEMS['qtd_SEMS'] = dfSEMS['qtd_SEMS'].str.replace('.', ',')      #troca virgula por ponto
dfsemsexp = pd.DataFrame()
dfsemsexp['qtd_produzida'] = dfSEMS['qtd_SEMS']
dfrelacao = pd.read_excel('relacaoUC.xlsx')
dfsemsexp = pd.merge(dfSEMS, dfrelacao)
dfsemsexp.rename(columns={'qtd_SEMS': 'qtd_produzida'}, inplace = True)
dfsemsexp['tip_origem'] =  'Gerada' #Cria coluna "tip_origem"
#dfsemsexp['qtd_produzida'] = dfSEMS['qtd_Solarview']
dfsemsexp = dfsemsexp[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida']] 
dfsemsexp.to_csv(f"mes_{mmes}_Geração_SEMS.csv", sep=";", index=False )

In [10]:
UCSOLARVIEW2 = ["24719", "24707", "24716", "24714", "328807", "328805", "328822", "328802", "328803", "Usina 10", "Usina 11", "Usina 12", "Usina 13", "Usina 14", "Usina 15"]
#UCSOLARVIEW2 = []

if UCSOLARVIEW2:    
    usuario = "thiago.lorena@holambra.com.br"
    senha ="@holambra"
    main_tab = logaSOLARVIEW(wait, navegador, usuario, senha)                                          #função que loga no site
    for uc in UCSOLARVIEW2:                                   #loop que passa pelas unidades e armazena no dfmain
        dfUC, vetor_erros = SOLARVIEW(uc, wait, navegador, main_tab, download_path, endexcel, vetor_erros, mode)
        dfUC = tratasview(dfUC, dmes)
        dfUC['dat_producao'] = datas                        #adiciona a coluna com as datas
        dfUC['UC'] = [uc]*dmes                              #adiciona a coluna com a UC
        dfsolarview2 = pd.concat([dfsolarview2, dfUC])      #adiciona dfUC no dfsolarview1

    dfsolarview = pd.concat([dfsolarview, dfsolarview2])    #armazena o dfsolarview2 no dfsolarview
else:
    print("Solarview2 vazio.")


dfsolarview = dfsolarview.fillna(0)
dfsolarview['qtd_Solarview'] = dfsolarview['qtd_Solarview'].astype(str)      #converte em string
dfsolarview['qtd_Solarview'] = dfsolarview['qtd_Solarview'].str.replace('.', ',')      #troca virgula por ponto
dfs1exp = pd.DataFrame()
dfs1exp['qtd_produzida'] = dfsolarview['qtd_Solarview']
dfrelacao = pd.read_excel('relacaoUC.xlsx')
dfs1exp = pd.merge(dfsolarview, dfrelacao)
dfs1exp.rename(columns={'qtd_Solarview': 'qtd_produzida'}, inplace = True)
dfs1exp['tip_origem'] =  'Gerada' #Cria coluna "tip_origem"
#dfs1exp['qtd_produzida'] = dfsolarview['qtd_Solarview']
dfs1exp = dfs1exp[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida']] 
dfs1exp.to_csv(f"mes_{mmes}_Geração_Solarview2.csv", sep=";", index=False )

 download da  24719 falhou 

 download da  24707 falhou 

 download da  24716 falhou 

 download da  24714 falhou 

 download da  328807 falhou 

 download da  328805 falhou 

 download da  328822 falhou 

 download da  328802 falhou 

 download da  328803 falhou 

 download da  Usina 10 falhou 

 download da  Usina 11 falhou 

 download da  Usina 12 falhou 

 download da  Usina 13 falhou 

 download da  Usina 14 falhou 

 download da  Usina 15 falhou 



KeyError: 'qtd_Solarview'

In [12]:
if UCSOLARVIEW3:    
    usuario = "jaco@liebe.com.br"
    senha ="12345678"
    main_tab = logaSOLARVIEW(wait, navegador, usuario, senha)                                          #função que loga no site
    for uc in UCSOLARVIEW3:                                   #loop que passa pelas unidades e armazena no dfmain
        try:
            dfUC, vetor_erros = SOLARVIEW(uc, wait, navegador, main_tab, download_path, endexcel, vetor_erros, mode)
        except:
            print("erro - jacob")
        dfUC = tratasview(dfUC, dmes)
        dfUC['dat_producao'] = datas                        #adiciona a coluna com as datas
        dfUC['UC'] = [uc]*dmes                              #adiciona a coluna com a UC
        dfsolarview3 = pd.concat([dfsolarview3, dfUC])      #adiciona dfUC no dfsolarview1

    dfsolarview = pd.concat([dfsolarview, dfsolarview3])    #armazena o dfsolarview1 no dfsolarview
else:
    print("Solarview3 vazio.")


dfsolarview = dfsolarview.fillna(0)
dfsolarview['qtd_Solarview'] = dfsolarview['qtd_Solarview'].astype(str)      #converte em string
dfsolarview['qtd_Solarview'] = dfsolarview['qtd_Solarview'].str.replace('.', ',')      #troca virgula por ponto
dfs1exp = pd.DataFrame()
dfs1exp['qtd_produzida'] = dfsolarview['qtd_Solarview']
dfrelacao = pd.read_excel('relacaoUC.xlsx')
dfs1exp = pd.merge(dfsolarview, dfrelacao)
dfs1exp.rename(columns={'qtd_Solarview': 'qtd_produzida'}, inplace = True)
dfs1exp['tip_origem'] =  'Gerada' #Cria coluna "tip_origem"
#dfs1exp['qtd_produzida'] = dfsolarview['qtd_Solarview']
dfs1exp = dfs1exp[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida']] 
dfs1exp.to_csv(f"mes_{mmes}_Geração_Solarview3.csv", sep=";", index=False )

NameError: name 'UCSOLARVIEW3' is not defined

In [ ]:
foxescloud = 0       #tirar do 1 para não rodar
#foxescloud = 1

if foxescloud == 1:     
    # define as credenciais de acesso à API
    f.username = "jacob.liebe"
    f.password = "solar123"
    f.device_sn = "J0RK753034EG007"

    d= f'{ymes}-{mmes}-{dmes}'
    api = f.get_report('month',d=d , v= 'generation') 
    data = api[0]['data']                               # Extrair a lista de dados da resposta
    dffoxc = pd.DataFrame(data)                             # Converter a lista de dados em um DataFrame
    dffoxc.columns = ['index', 'qtd_Solarview']                # Nomear as colunas do DataFrame
    lz = pd.Series([0]* (dmes - len(dffoxc)))       # Crie um Series com linhas contendo zeros
    dffoxc = pd.concat([dffoxc, lz], ignore_index=True)   #adiciona os zeros no novo_df
    dffoxc['dat_producao'] = datas                       #adiciona a coluna com as datas
    dffoxc['UC'] = [230513]*dmes                        #adiciona a coluna com a UC
    dffoxc = dffoxc[['dat_producao', 'UC', 'qtd_Solarview']]

    dfsolarview = pd.concat([dfsolarview, dffoxc])
else:
    print("Foxescloud vazio.")

In [10]:
#----------------------------------------------------------------------//organização do dataframe e salva em um arquivo CSV//----------------------------------------------------------------------
dfmain = dfSEMS.merge(dfsolarview, on=['dat_producao', 'UC'], how='outer')         #vamos trabalhar com apenas: 'dat_producao', 'UC', 'qtd_produzida' e faer um merge das duas colunas de ref
#dfmain = dfsolarview
dfmain = dfmain.fillna(0)                                       #preenche os "Nan" com zeros
dfmain['qtd_produzida'] = dfmain['qtd_SEMS'] + dfmain['qtd_Solarview']                      #soma as colunas de UCs fiferentes
dfmain['qtd_produzida'] = dfmain['qtd_produzida'].astype(str)      #converte em string
dfmain['qtd_produzida'] = dfmain['qtd_produzida'].str.replace('.', ',')      #troca virgula por ponto
dfmain['qtd_Solarview'] = dfmain['qtd_Solarview'].astype(str)      #converte em string
dfmain['qtd_Solarview'] = dfmain['qtd_Solarview'].str.replace('.', ',')      #troca virgula por ponto


#----------------------------------------------------------------------//#Final organiza e cria os CSV's //----------------------------------------------------------------------
dfrelacao = pd.read_excel('relacaoUC.xlsx')   #df que armazena a relaçãocod_ponto_medicao(UC) cod_agente e cod_unidade (le o excel chamado "relacaoUC.xlsx" )

#df com os dados:
dfmain = pd.merge(dfmain, dfrelacao)          #faze um merge (tipo um procv) adiciona as colunas: 'cod_agente', 'cod_unidade', 'cod_ponto_medicao'
dfmain['tip_origem'] =  'Gerada'              #Cria coluna "tip_origem"
dfmain = dfmain[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida', 'qtd_Solarview']]      #organiza as colunas

#compara com os dados do dia anterior:
if antigo!='' :
    dfantigo = pd.read_csv(f"{antigo}_Geração.csv",sep=';') 
    dfantigo = dfantigo[['cod_ponto_medicao', 'dat_producao', 'qtd_produzida']]         #deixa apenas as colunas importantes
    dfmain = pd.merge(dfantigo, dfmain, on=['cod_ponto_medicao', 'dat_producao'], how='outer', suffixes=('', '_antigo'))   #merge para ter a qtd_produzida duplicada
    def comparac(coluna1, coluna2):
        return '' if coluna1 == coluna2 else 'sim'
    dfmain['alterou'] = dfmain.apply(lambda row: comparac(row['qtd_produzida'], row['qtd_produzida_antigo']), axis=1) #cria uma coluna com a informação "sim" caso tenha dados diferentes
    dfmain= dfmain[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida', 'qtd_produzida_antigo','alterou']]

#df com os erros:
dferros_df = pd.merge(dferros_df, dfrelacao)      #faze um merge (tipo um procv) adiciona as colunas: 'cod_agente', 'cod_unidade', 'cod_ponto_medicao'
dferros_df['tip_origem'] =  'Gerada'              #Cria coluna "tip_origem"
dferros_df = dferros_df[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida']]      #organiza as colunas

#cria os CSV's:
if mode ==0:
    dfmain.to_csv(f"{datatual}_Geração.csv", sep=";", index=False )
    dferros_df.to_csv(f"{datatual}_Erros_Geração.csv", sep=";", index=False )
else:
    dfmain.to_csv(f"mes_{mmes}_Geração.csv", sep=";", index=False )
    dferros_df.to_csv(f"mes_{mmes}_Erros_Geração.csv", sep=";", index=False )

KeyError: 'qtd_Solarview'

In [ ]:
if foxescloud == 1:     
    # define as credenciais de acesso à API
    f.username = "jacob.liebe"
    f.password = "solar123"
    f.device_sn = "J0RK753034EG007"

    d= f'{ymes}-{mmes}-{dmes}'
    api = f.get_report('month',d=d , v= 'generation') 
    data = api[0]['data']                               # Extrair a lista de dados da resposta
    dffoxc = pd.DataFrame(data)                             # Converter a lista de dados em um DataFrame
    dffoxc.columns = ['index', 'qtd_Solarview']                # Nomear as colunas do DataFrame
    lz = pd.Series([0]* (dmes - len(dffoxc)))       # Crie um Series com linhas contendo zeros
    dffoxc = pd.concat([dffoxc, lz], ignore_index=True)   #adiciona os zeros no novo_df
    dffoxc['dat_producao'] = datas                       #adiciona a coluna com as datas
    dffoxc['UC'] = [230513]*dmes                        #adiciona a coluna com a UC
    dffoxc = dffoxc[['dat_producao', 'UC', 'qtd_Solarview']]

    dfsolarview = pd.concat([dfsolarview, dffoxc])
else:
    print("Foxescloud vazio.")

#----------------------------------------------------------------------//7-Setima parte: Canadian Solar//----------------------------------------------------------------------
UCANADIANSOLAR = [99807, 35198605, 5726700, 31383823, 31383807, 33112282, 18459960, 28158601, 18444377]
vetor_erros += UCANADIANSOLAR   #adiciona as UC's no vetor_erros

#----------------------------------------------------------------------//adicional-preenche dataframe com os erros//----------------------------------------------------------------------
if vetor_erros:  
    for uc in vetor_erros: 
        UCS = [uc]*dmes
        dfUC = pd.DataFrame({'UC': UCS}) 
        dfUC['qtd_produzida'] = 0  
        dfUC['dat_producao'] = datas                                       #loop que passa pelas unidades e armazena no dfmain
        dferros_df = pd.concat([dferros_df, dfUC])
else:
    print("sem erros.")


#----------------------------------------------------------------------//organização do dataframe e salva em um arquivo CSV//----------------------------------------------------------------------
dfmain = dfSEMS.merge(dfsolarview, on=['dat_producao', 'UC'], how='outer')         #vamos trabalhar com apenas: 'dat_producao', 'UC', 'qtd_produzida' e faer um merge das duas colunas de ref
dfmain = dfmain.fillna(0)                                       #preenche os "Nan" com zeros
dfmain['qtd_produzida'] = dfmain['qtd_SEMS'] + dfmain['qtd_Solarview']                      #soma as colunas de UCs fiferentes
dfmain['qtd_produzida'] = dfmain['qtd_produzida'].astype(str)      #converte em string
dfmain['qtd_produzida'] = dfmain['qtd_produzida'].str.replace('.', ',')      #troca virgula por ponto
dfmain['qtd_Solarview'] = dfmain['qtd_Solarview'].astype(str)      #converte em string
dfmain['qtd_Solarview'] = dfmain['qtd_Solarview'].str.replace('.', ',')      #troca virgula por ponto


#----------------------------------------------------------------------//#Final organiza e cria os CSV's //----------------------------------------------------------------------
dfrelacao = pd.read_excel('relacaoUC.xlsx')   #df que armazena a relaçãocod_ponto_medicao(UC) cod_agente e cod_unidade (le o excel chamado "relacaoUC.xlsx" )

#df com os dados:
dfmain = pd.merge(dfmain, dfrelacao)          #faze um merge (tipo um procv) adiciona as colunas: 'cod_agente', 'cod_unidade', 'cod_ponto_medicao'
dfmain['tip_origem'] =  'Gerada'              #Cria coluna "tip_origem"
dfmain = dfmain[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida', 'qtd_SEMS', 'qtd_Solarview']]      #organiza as colunas

#compara com os dados do dia anterior:
if antigo!='' :
    dfantigo = pd.read_csv(f"{antigo}_Geração.csv",sep=';') 
    dfantigo = dfantigo[['cod_ponto_medicao', 'dat_producao', 'qtd_produzida']]         #deixa apenas as colunas importantes
    dfmain = pd.merge(dfantigo, dfmain, on=['cod_ponto_medicao', 'dat_producao'], how='outer', suffixes=('', '_antigo'))   #merge para ter a qtd_produzida duplicada
    def comparac(coluna1, coluna2):
        return '' if coluna1 == coluna2 else 'sim'
    dfmain['alterou'] = dfmain.apply(lambda row: comparac(row['qtd_produzida'], row['qtd_produzida_antigo']), axis=1) #cria uma coluna com a informação "sim" caso tenha dados diferentes
    dfmain= dfmain[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida', 'qtd_produzida_antigo','alterou']]

#df com os erros:
dferros_df = pd.merge(dferros_df, dfrelacao)      #faze um merge (tipo um procv) adiciona as colunas: 'cod_agente', 'cod_unidade', 'cod_ponto_medicao'
dferros_df['tip_origem'] =  'Gerada'              #Cria coluna "tip_origem"
dferros_df = dferros_df[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida']]      #organiza as colunas

#cria os CSV's:
if mode ==0:
    dfmain.to_csv(f"{datatual}_Geração.csv", sep=";", index=False )
    dferros_df.to_csv(f"{datatual}_Erros_Geração.csv", sep=";", index=False )
else:
    dfmain.to_csv(f"mes_{mmes}_Geração.csv", sep=";", index=False )
    dferros_df.to_csv(f"mes_{mmes}_Erros_Geração.csv", sep=";", index=False )

NameError: name 'foxescloud' is not defined

In [ ]:
# dfm = pd.DataFrame()     
# dfantigo = pd.read_csv("30-12-23_Geração.csv",sep=';') 
# dfantigo = dfantigo[['cod_ponto_medicao', 'dat_producao', 'qtd_produzida']]         #deixa apenas as colunas importantes
# dfm = pd.read_csv("31-12-23_Geração.csv",sep=';')
# dfm = dfm[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida']]
# dfm = pd.merge(dfantigo, dfm, on=['cod_ponto_medicao', 'dat_producao'], how='outer', suffixes=('', '_antigo'))   #merge para ter a qtd_produzida duplicada
# dfm['alterou'] = dfm.apply(lambda row: comparac(row['qtd_produzida'], row['qtd_produzida_antigo']), axis=1) #cria uma coluna com a informação "sim" caso tenha dados diferentes
# dfm= dfm[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida', 'qtd_produzida_antigo','alterou']]

In [ ]:
#teste para criar csv com os erros
vetor_erros = []
datas, dmes, datatual= datasmes(mode)
UCANADIANSOLAR = [99807, 35198605, 5726700, 31383823, 31383807, 33112282, 18459960, 28158601, 18444377]
vetor_erros += UCANADIANSOLAR 
dferros_df = pd.DataFrame()

#----------------------------------------------------------------------//adicional-preenche dataframe com os erros//----------------------------------------------------------------------
if vetor_erros:  
    for uc in vetor_erros: 
        UCS = [uc]*dmes
        dfUC = pd.DataFrame({'UC': UCS}) 
        dfUC['qtd_produzida'] = 0  
        dfUC['dat_producao'] = datas                                       #loop que passa pelas unidades e armazena no dfmain
        dferros_df = pd.concat([dferros_df, dfUC])
else:
    print("sem erros.")

#cria excel com os erros:
dfrelacao = pd.read_excel('relacaoUC.xlsx')   #df que armazena a relaçãocod_ponto_medicao(UC) cod_agente e cod_unidade (le o excel chamado "relacaoUC.xlsx" )
dferros_df = pd.merge(dferros_df, dfrelacao)      #faze um merge (tipo um procv) adiciona as colunas: 'cod_agente', 'cod_unidade', 'cod_ponto_medicao'
dferros_df['tip_origem'] =  'Gerada'              #Cria coluna "tip_origem"
dferros_df = dferros_df[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida']]      #organiza as colunas
dferros_df.to_csv(f"{datatual}_Erros_Geração.csv", sep=";", index=False )

In [ ]:
#Salva Excel quando só tem dados do sems

dfmain = dfSEMS
dfmain = dfmain.fillna(0)                                       #preenche os "Nan" com zeros
dfmain['qtd_produzida'] = dfmain['qtd_SEMS']                      #soma as colunas de UCs fiferentes
dfmain['qtd_produzida'] = dfmain['qtd_produzida'].astype(str)      #converte em string
dfmain['qtd_produzida'] = dfmain['qtd_produzida'].str.replace('.', ',')      #troca virgula por ponto


dfrelacao = pd.read_excel('relacaoUC.xlsx')   #df que armazena a relaçãocod_ponto_medicao(UC) cod_agente e cod_unidade (le o excel chamado "relacaoUC.xlsx" )
dfmain = pd.merge(dfmain, dfrelacao)          #faze um merge (tipo um procv) adiciona as colunas: 'cod_agente', 'cod_unidade', 'cod_ponto_medicao'
dfmain['tip_origem'] =  'Gerada'              #Cria coluna "tip_origem"
dfmain = dfmain[['cod_agente', 'cod_unidade', 'cod_ponto_medicao', 'dat_producao', 'tip_origem', 'qtd_produzida', 'qtd_SEMS']]      #organiza as colunas
dfmain.to_csv(f"{datatual}_Geração.csv", sep=";", index=False )
dferros_df.to_csv(f"{datatual}_Erros.csv", sep=";", index=False )

In [ ]:
#compara o csv antico com o csv novo
antigo="28-12-23" 
novo = "29-12-23" 
dfantigo = pd.read_csv(f"{antigo}_Geração.csv",sep=';') 
dfnovo = pd.read_csv(f"{novo}_Geração.csv",sep=';') 

dfnovo = pd.merge(dfnovo, dfantigo, on=['cod_ponto_medicao', 'dat_producao'], how='outer', suffixes=('', '_antigo'))
def comparac(coluna1, coluna2):
    return '' if coluna1 == coluna2 else 'sim'

dfnovo['alterou'] = dfnovo.apply(lambda row: comparac(row['qtd_produzida'], row['qtd_produzida_antigo']), axis=1)

dfnovo= dfnovo[['cod_ponto_medicao', 'dat_producao','qtd_produzida', 'qtd_produzida_antigo','alterou']]

dfnovo.to_csv(f"{novo}_Comparação.csv", sep=";", index=False )